# Задания для рефакторинга

1. Сделать все гибко

2. Сделать все красиво

3. Сделать все просто

4. Сделать все логично


Кобзарь Олег, Хабибуллин Ринат, Буденный Семен, Адрианова Алла, Логинов Александр, Лемихов Александр
# Обработчик данных 

В данной тетрадке собраны все средства для работы с данными (пре- и пост- процессор; генерация входных данных для модели, анализ рассчитанных значений, построение графиков)

Большое количество кода вынесено в модуле - здесь в основном управление

`well_name` - номер исследуемой скважины, например `1`

Исходными данные лежат в папке  `data` в папках, соответствующим названиям скважин. Это данные
1. Со станции управления (`data/well_name/well_name.csv`). Большое количество записей, которые нужно преобразовать в "шахмоткоподобный" вид для дальнейшей работы. Данные со СУ могут быть как высокочастотными (токи, напряжения), так и низкочастотными. 
2. С шахматки `data/well_name/Скв. well_name (01.07.2018-31.03.2019).xls`. Низкочастотные данные. 
3. C техрежима `data/tr/Техрежим, , февраль 2019.xls`. Какой насос спущен, ПЭД, на какую глубину. Эти данные постоянны на всем периоде расчета. В одном файлике один месяц и данные для всех скважин.



## Общий workflow
Здесь кратко, подробное описание будет над ячейками.
1. Обработка исходных данных со СУ и приведение их в удобный формат. (init_edit)
2. Считавание подготовленных ранее данных со СУ, шахматки, а затем генерации входных данных для модели, конкретно, адаптации модели скважины за выбранный период времени. (adaptation_input)
3. Адаптация. Теперь можно открыть файл processor.py и в нем, выставив настройки и определив входные данные, запустить расчет скважину. После адаптации данные появится данные с калибровочными коэффициентами. (adaptation)
4. Считывание рассчитанных значений адаптации, данных со СУ и шахматки. Генерации файлов для восстановления дебитов (restore_input)
5. Восстановление. Работа в processor.py, где необходимо изменить метод расчета и указать на новые входные данные. (restore) 
6. Заключительный анализ: сведение результатов адаптации и восстановления; расчет различных метрик для оценки метода (тоже в папке restore)

По идее, если данные уже сгенерированы, каждый пункт может повторяться многократно

Импорт модулей. Некоторые функции и методы вынесены в отдельные `.py`
* `preprocessor` - для обработки данных в и из модели
* `processor` - для непосредственного запуска расчета адаптации или восстановления
* `postprocessor` - для завершающего анализа сгенерированных данных. Определение метрик успешности модели, паттернов, событий, зависимостей.
* `plotly_workflow` - для быстрого построения шаблонизированных графиков

In [ ]:
import os
import sys
sys.path.append('../../../')
import pandas as pd
import datetime
from multiprocessing import Pool

In [ ]:
import plot_workflow.plotly_option as pltl_opt
import plot_workflow.plotly_workflow as pltl_wf

from preproc_p import workflow_cs_data
from preproc_p import workflow_chess_data
from preproc_p import preproc_tool
from preproc_p import workflow_calc_data
from preproc_p import workflow_tr_data
from preproc_p import filtration
from proc_p import processor as proc

from ml import calibr_restore as calibr_restore
from postproc_p import result_and_metrics as result_and_metrics

Общие настройки и флаги для работы. Указание номера скважины и название файла шахматки. Флаги для запуска тех или иных ячеек

In [ ]:
well_name = '3922'
chess_file_name = f'Скв. {well_name} (01.01.2020-29.02.2020).xls'
# TODO убрать флаги
read_initial_data = True
plot_initial_data = True
create_input_data = True
auto_open_html = True
multiprocessing_on = True #TODO может быть убрать флаги, как лучше?

In [ ]:
static_data_full_path = preproc_tool.find_full_path_by_pattern(os.getcwd(), "*static_data.xlsx*")[0]

Определение путей к данным. Создание метки времени к генерируемым папкам. (чтоб данные не перезаписывались)

In [ ]:
current_path = os.getcwd()
time_mark = '' #datetime.datetime.today().strftime('%Y_%m_%d_%H_%M_%S')
path_to_data = current_path + "\\data\\"
path_to_work_dir = current_path + "\\data\\" + well_name +  "\\"
save_dir_name = 'init_edit'
path_to_save = path_to_work_dir + save_dir_name + '\\'
dirnames_list = []
for (dirpath, dirnames, filenames) in os.walk(path_to_data):
    dirnames_list.extend(dirnames)
    break
print(dirnames_list)
dynamic_data_full_path = path_to_save + well_name + "_first_edit.csv"

Задание интервалов, на которых будет вестить расчет. Их может быть несколько. Рекомендуется поставить малый период времени, выявить ошибки, а затем запускать на месяцы и годы

In [ ]:
#left_boundary = [datetime.datetime(2018,8,1), datetime.datetime(2018,11,29)]
#right_boundary = [datetime.datetime(2018,11,5), datetime.datetime(2019,2,28)]
#left_boundary = [datetime.datetime(2018,8,3), datetime.datetime(2018,11,29),   datetime.datetime(2019,2,19)]
#right_boundary = [datetime.datetime(2018,11,6), datetime.datetime(2019,2,4),  datetime.datetime(2019,2,28)]
#left_boundary = [datetime.datetime(2019,1,30)]
#right_boundary = [datetime.datetime(2019,2,28)]
left_boundary = [datetime.datetime(2018,6,27)]
right_boundary = [datetime.datetime(2020,7,27)]

Определение функции которая все запустит в многопотоке и произведет из данного ноутбука адаптацию и восстановление. (Временный костыль - работает только здесь, в модуле не может)

In [ ]:
def run_calculation(thread_option_list):
    if __name__ == '__main__':
        with Pool(amount_of_threads) as p:
            p.map(proc.calc,
                  thread_option_list)

In [ ]:
global_names = preproc_tool.GlobalNames()

Учет ВСП

In [ ]:
well_stops_path = preproc_tool.find_full_path_by_pattern(os.getcwd(), "*Вынгаяхинское ВСП*")[0]

In [ ]:
well_stops = pd.read_csv(well_stops_path, delimiter = ';', engine = 'python')
well_stops = well_stops[well_stops['Скв'] == int(well_name)]

In [ ]:
well_stops_start = well_stops['ДатаСтарта'].values
well_stops_end = well_stops['Дата_Окончания'].values

In [ ]:
for i in range(len(well_stops_start)):
    well_stops_start[i] = pd.to_datetime(well_stops_start[i])

In [ ]:
for i in range(len(well_stops_end)):
    well_stops_end[i] = pd.to_datetime(well_stops_end[i])

In [ ]:
well_stops_start

In [ ]:
well_stops_end

In [ ]:
left_boundary = list(well_stops_start)
right_boundary = list(well_stops_end)

# 1. Обработка исходных данных
Чтение исходных данных `well_name.csv` со СУ и преобразование их в удобный формат. Процесс небыстрый из-за несовершенства алгоритма форматирования и количества данных (около 1-1,5 млн. записей)

In [ ]:
#%%time
if read_initial_data:
    try:
        os.mkdir(path_to_work_dir + save_dir_name)
    except:
        pass

### Работа с данными ГРАД

In [ ]:
grad_data_full_path = preproc_tool.find_full_path_by_pattern(os.getcwd(), f"*{well_name}.csv*")[0]

In [ ]:
grad_data = workflow_cs_data.read_and_format_good_tm_data(well_name, grad_data_full_path)

In [ ]:
grad_data.to_csv(path_to_save + well_name + "_first_edit_grad.csv")

In [ ]:
well_traces_grad = pltl_wf.create_traces_list_for_all_columms(grad_data, 
                                                         'lines+markers',
                                                         use_gl = True)
pltl_wf.plot_subplots(well_traces_grad,  path_to_save + well_name + "_first_edit_grad.html", two_equal_subplots = True, auto_open = auto_open_html)

### Работа с данными со СУ

In [ ]:
cs_data_full_path = preproc_tool.find_full_path_by_pattern(os.getcwd(), f"*{well_name}*", 'флэш')[0]

In [ ]:
cs_data = workflow_cs_data.read_and_format_bad_tm_data(cs_data_full_path)

In [ ]:
cs_data.to_csv(path_to_save + well_name + "_first_edit_cs.csv")

In [ ]:
well_traces_cs = pltl_wf.create_traces_list_for_all_columms(cs_data, 
                                                         'lines+markers',
                                                         use_gl = True)
pltl_wf.plot_subplots(well_traces_cs,  path_to_save + well_name + "_first_edit_cs.html", two_equal_subplots = True, auto_open = auto_open_html)

### Работа с данными шахматки

In [ ]:
chess_data = workflow_chess_data.load_and_edit_chess_data(path_to_work_dir + chess_file_name,
                                         '1d', without_changing = True)

In [ ]:
chess_data.to_csv(path_to_save + well_name + "_first_edit_chess.csv")

In [ ]:
well_traces_chess = pltl_wf.create_traces_list_for_all_columms(chess_data, 
                                                         'lines+markers',
                                                         use_gl = True)

pltl_wf.plot_subplots(well_traces_chess,  path_to_save + well_name + "_first_edit_chess.html", two_equal_subplots = True, auto_open = auto_open_html)

Построение графиков для нескольких интересующих параметров по исходным данных со СУ в `well_name__first_edit_report.html`. Процесс занимает около 3-5 минут.

### Объединение данных в единый динамический DataFrame

переименование нужных параметров, добавление меток источников данных

In [ ]:
grad_data_edited = pd.read_csv(
    path_to_save + well_name + "_first_edit_grad.csv", index_col='Время', parse_dates=True, dayfirst = True)

In [ ]:
grad_data_edited = preproc_tool.rename_columns_by_dict(grad_data_edited)
grad_data_edited = preproc_tool.solve_dimensions(grad_data_edited)
grad_data_edited = preproc_tool.mark_df_columns(grad_data_edited, "ГРАД")

In [ ]:
cs_data_edited = pd.read_csv(
    path_to_save + well_name + "_first_edit_cs.csv", index_col='Время', parse_dates=True, dayfirst = True)

In [ ]:
cs_data_edited = preproc_tool.rename_columns_by_dict(cs_data_edited)
cs_data_edited = preproc_tool.solve_dimensions(cs_data_edited)
cs_data_edited = preproc_tool.mark_df_columns(cs_data_edited, "СУ")

In [ ]:
chess_data_edited = pd.read_csv(
    path_to_save + well_name + "_first_edit_chess.csv", index_col='Время', parse_dates=True, dayfirst = True)

In [ ]:
chess_data_edited = preproc_tool.rename_columns_by_dict(chess_data_edited)
chess_data_edited = preproc_tool.solve_dimensions(chess_data_edited)
chess_data_edited = preproc_tool.mark_df_columns(chess_data_edited, "ШТР")

In [ ]:
dynamic_data = grad_data_edited.copy()

In [ ]:
dynamic_data = dynamic_data.join(cs_data_edited, how = 'outer')

In [ ]:
dynamic_data = dynamic_data.join(chess_data_edited, how = 'outer')

In [ ]:
dynamic_data = dynamic_data.sort_index()

In [ ]:
def solve_dim(values, critical_value, multyplier):
    new_values = []
    for i in values:
        if i > critical_value:
            this_value = i * multyplier
            new_values.append(this_value)
        else:
            new_values.append(i)
    return new_values

In [ ]:
dynamic_data[global_names.p_lin_atm + " (ГРАД)"] = solve_dim(dynamic_data[global_names.p_lin_atm + " (ГРАД)"].values, 40, 0.1)

In [ ]:
dynamic_data[global_names.cos_phi_d + " (ГРАД)"] = solve_dim(dynamic_data[global_names.cos_phi_d + " (ГРАД)"].values, 1.1, 0.01)

In [ ]:
dynamic_data.to_csv(path_to_save + well_name + "_first_edit.csv")

In [ ]:
well_traces_dynamic_data = pltl_wf.create_traces_list_for_all_columms(dynamic_data, 
                                                         'lines+markers',
                                                         use_gl = True)

pltl_wf.plot_subplots(well_traces_dynamic_data,  path_to_save + well_name + "_first_edit.html", two_equal_subplots = True, auto_open = auto_open_html)

## Возможно надо сделать репорты для СУ и ШТР

# 2. Вторичная обработка исходных данных для использования в модели

Для данного этапа уже нужные обработанные данные со станции управления.

Чтение исходных данных со СУ (в уже подготовленных). Обработка этих данных - ресемпл, пометка названий столбцов с помощью `(СУ)`. Т.к. данные высокочастотные, осредним их по 3 часам - примерному времени замера дебита скважины на АГЗУ. Обработка будет отличаться в зависимости от того, какие данные вы генерируете - для адаптации выбрасываются интервалы без замеров дебитов, для восстановления - нет

In [ ]:
dynamic_data = pd.read_csv(dynamic_data_full_path, index_col = 'Время', parse_dates = True)
prepared_dynamic_data = dynamic_data.copy()

In [ ]:
preproc_tool.find_column_in_df_columns(dynamic_data, 'D')

In [ ]:
global_names.chosen_q_liq_m3day = global_names.q_liq_m3day + ' (ГРАД)'
global_names.chosen_watercut_perc = global_names.watercut_perc + ' (ГРАД)'
global_names.chosen_gor_m3m3 = None

global_names.chosen_p_buf_atm = global_names.p_lin_atm + ' (ГРАД)'
global_names.chosen_p_intake_atm = global_names.p_intake_atm + ' (ГРАД)'
global_names.chosen_t_intake_c =  global_names.t_intake_c + ' (ГРАД)'

global_names.chosen_d_choke_mm = 'Dшт (ШТР)'

global_names.chosen_cos_phi_d = global_names.cos_phi_d + ' (ГРАД)'
global_names.chosen_i_a_motor_a = global_names.i_a_motor_a + ' (ГРАД)'
global_names.chosen_u_motor_v = None
global_names.chosen_motor_load_perc = global_names.motor_load_perc + ' (ГРАД)'
global_names.chosen_freq_hz = global_names.freq_hz + ' (ГРАД)'
global_names.chosen_active_power_kwt = global_names.active_power_kwt + ' (ГРАД)'

In [ ]:
global_names.chosen_gor_m3m3 = global_names.gor_m3m3
global_names.chosen_d_choke_mm =  global_names.d_choke_mm
global_names.chosen_u_motor_v =  global_names.u_motor_v

In [ ]:
prepared_dynamic_data[global_names.q_liq_m3day] = prepared_dynamic_data[global_names.chosen_q_liq_m3day]
prepared_dynamic_data[global_names.watercut_perc] = prepared_dynamic_data[global_names.chosen_freq_hz] * 0 + 85

prepared_dynamic_data[global_names.p_buf_atm] = prepared_dynamic_data[global_names.chosen_p_buf_atm]
prepared_dynamic_data[global_names.p_intake_atm] = prepared_dynamic_data[global_names.chosen_p_intake_atm]
prepared_dynamic_data[global_names.t_intake_c] = prepared_dynamic_data[global_names.chosen_t_intake_c]

prepared_dynamic_data[global_names.d_choke_mm] = prepared_dynamic_data[global_names.chosen_freq_hz] * 0 + 32

prepared_dynamic_data[global_names.cos_phi_d] = prepared_dynamic_data[global_names.chosen_cos_phi_d]
prepared_dynamic_data[global_names.i_a_motor_a] = prepared_dynamic_data[global_names.chosen_i_a_motor_a]
prepared_dynamic_data[global_names.motor_load_perc] = prepared_dynamic_data[global_names.chosen_motor_load_perc]
prepared_dynamic_data[global_names.freq_hz] = prepared_dynamic_data[global_names.chosen_freq_hz]
prepared_dynamic_data[global_names.active_power_kwt] = prepared_dynamic_data[global_names.chosen_active_power_kwt]

In [ ]:
prepared_dynamic_data[global_names.u_motor_v] = prepared_dynamic_data[global_names.active_power_kwt] * 1000 / \
                                                prepared_dynamic_data[global_names.i_a_motor_a + ' (ГРАД)']

In [ ]:
prepared_dynamic_data[global_names.gor_m3m3] = prepared_dynamic_data[global_names.chosen_freq_hz] * 0 + 186

In [ ]:
prepared_dynamic_data[global_names.gor_m3m3] = (prepared_dynamic_data[global_names.q_gas_m3day +' (ГРАД)']   /
                        (prepared_dynamic_data[global_names.q_liq_m3day] * (1 - prepared_dynamic_data[global_names.watercut_perc] / 100)))

## Тут должна быть фильтрация, удаление мусора из колонок

In [ ]:
all_banches = pltl_opt.create_banches_for_report(report_type = 'second_edit_data')
pltl_wf.create_report_html(prepared_dynamic_data, all_banches, path_to_save + well_name + "_second_edit_report.html",  auto_open = auto_open_html)

In [ ]:
prepared_dynamic_data.to_csv(path_to_save + well_name + "_second_edit.csv")

In [ ]:
well_traces_prepared_dynamic_data = pltl_wf.create_traces_list_for_all_columms(prepared_dynamic_data, 
                                                         'lines+markers',
                                                         use_gl = True)

pltl_wf.plot_subplots(well_traces_prepared_dynamic_data,  path_to_save + well_name + "_second_edit.html", two_equal_subplots = True, auto_open = auto_open_html)

## Тут общий ресемпл (возможно нет)

# 3. Подготовка данных к адаптации

In [ ]:
time_to_resamle = '1h'
input_data_dir_name = 'adapt_input_' + time_mark
path_to_input_data = path_to_work_dir + input_data_dir_name + '\\'
plot_file_path = path_to_input_data + well_name
if create_input_data:
    try:
        os.mkdir(path_to_work_dir + input_data_dir_name)
    except:
        pass

In [ ]:
prepared_dynamic_data = pd.read_csv(path_to_save + well_name + "_second_edit.csv", index_col = 'Время', parse_dates = True, dayfirst = True)

In [ ]:
adapt_input_dynamic_data = prepared_dynamic_data.resample(time_to_resamle).mean()

In [ ]:
adapt_input_dynamic_data[global_names.watercut_perc] = adapt_input_dynamic_data[global_names.watercut_perc].interpolate()

In [ ]:
adapt_input_dynamic_data = adapt_input_dynamic_data.dropna(subset = [global_names.q_liq_m3day])

In [ ]:
adapt_input_dynamic_data = adapt_input_dynamic_data.dropna(subset = [global_names.p_intake_atm])

Тут не должно быть фильтрации, но должен быть чек

In [ ]:
adapt_input_dynamic_data = filtration.get_filtred_by_sigma(adapt_input_dynamic_data, global_names.q_liq_m3day)

In [ ]:
adapt_input_dynamic_data = preproc_tool.cut_df(adapt_input_dynamic_data, left_boundary, right_boundary)

In [ ]:
adapt_input_dynamic_data = preproc_tool.fill_input_data(adapt_input_dynamic_data, global_names.return_essential_parameters())

In [ ]:
all_banches = pltl_opt.create_banches_for_report(report_type = 'second_edit_data')
pltl_wf.create_report_html(adapt_input_dynamic_data, all_banches, plot_file_path + "_adapt_input_report.html",  auto_open = auto_open_html)

In [ ]:
well_traces_adapt_input_dynamic_data = pltl_wf.create_traces_list_for_all_columms(adapt_input_dynamic_data, 
                                                         'lines+markers',
                                                         use_gl = True)
pltl_wf.plot_subplots(well_traces_adapt_input_dynamic_data,  plot_file_path + "_adapt_input.html", two_equal_subplots = True, auto_open = auto_open_html)

In [ ]:
adapt_input_dynamic_data.to_csv(path_to_input_data + well_name + '_adapt_input.csv')

Генерация входных данных для адаптации. Сведение данных с шахматки и со СУ. Создание папки с названием `adapt_input_` и временной пометкой и помещение в нее:
* сгенерированных входных данных для модели `well_name_adapt_input.csv`

Все входные данные на графике `well_name_adapt_input.html` в той же папке

Построение кривых в отчете только для тех данных, которые будут использоваться в модели в `well_name_adapt_input_report.html` в той же папке в стандартном виде. Можно настроить, какие данные выводить.

# 4. Адаптация
Нужно, используя сгенерированные данные `well_name_adapt_input` получить значения коэффициентов калибровок по напору и мощности с помощью `processor.py`

Подготовка к параллельному расчету.

В проекте UniflocVBA нужно вручную размножить надстройки до нужного количества потоков - каждая надстройка будет работать параллельно и рассчитывать определенную часть общих данных. 

По умолчанию выставлено 4 потока на 4 надстройках `UniflocVBA_7.xlam`, `UniflocVBA_7_1.xlam`, `UniflocVBA_7_2.xlam`, `UniflocVBA_7_3.xlam`. При желании можно добавить еще, не забыв изменить также номера потоков и их общее количество.

Группировка информации о потоках в единый список `thread_option_list`

In [ ]:
# настройка многопоточности
amount_of_threads = 12
dir_name_with_input_data = 'adapt_input_'

thread_option_list =proc.create_thread_list(well_name, dir_name_with_input_data, static_data_full_path,
                       amount_of_threads)

Запуск процесса адаптации - получения калибровок. Процесс может быть долгим - в среднем расчет идет медленнее, чем при восстановлении дебитов. 

Рассчет 1 месяца в среднем занимает около 15-20 минут.

Стоит отметить, что для аварийного выхода нужно вручную закрыть надстройки Excel (окна, которые открыты и в них ведется работа). После закрытия всех четырех процесс остановится и возникнет ошибка, контроль над jupyter notebook вернется и можно будет работать.

Поэтому для тщательной отладки `processor.py` рекомендуется запускать отдельно (лучше через PyCharm).

In [ ]:
%%time
run_calculation(thread_option_list)

# 5. Загрузка и анализ адаптации
После успешной адаптации нужно проанализировать ее корректность, построив графики. Для этого нужно указать директории с 
* результатами адапатации - `adaptation`  
* входными данными для адаптации - `adapt_input`

In [ ]:
dir_name_with_input_data = 'adapt_input_' + '\\'
input_data_file_name = well_name + '_adapt_input'
dir_name_with_calculated_data = 'adaptation_' + '\\'
calculated_data_file_name = well_name + '_adapt_1'

Если расчет проводился в многопоточном варианте, то загрузка будет отличаться. В папке с расчетами `adaptation` в сабдиректории `multiprocessing` будут результаты по частям в разных файликах, поэтому их нужно собрать в один файлик и поместить на уровень выше в `adaptation`. Если расчет был без использования многопоточности, этот шаг можно пропустить.

In [ ]:
if multiprocessing_on == True:
    first_result_data = preproc_tool.combine_multiprocessing_result(path_to_work_dir, dir_name_with_calculated_data)
    first_result_data.to_csv(path_to_work_dir + dir_name_with_calculated_data + calculated_data_file_name + '.csv')

Загрузка данных адаптации после расчета - в папке `adaptation_time_mark`  файл `well_name_adapt_1.csv`

In [ ]:
calculated_data = workflow_calc_data.load_calculated_data_from_csv(path_to_work_dir + dir_name_with_calculated_data +
                                                calculated_data_file_name +  '.csv', "ADAPT")

Чтение входных данных для адаптации из папки `adaptation_input` и объединение данных адатации и входных данных для нее.
Сохранение всех данных в папке `adaptation_time_mark` с названием `well_name_calc_and_input.csv`

In [ ]:
input_data = pd.read_csv(path_to_work_dir + dir_name_with_input_data + input_data_file_name +  '.csv',index_col = 'Время', parse_dates = True)
all_data = input_data.join(calculated_data, how = 'outer')

In [ ]:
if pd.isna(all_data.iloc[0][0]):
    print('Удаление пустой первой строки')
    all_data = all_data.drop(index = all_data.index[0])

In [ ]:
all_data.to_csv(path_to_work_dir + dir_name_with_calculated_data + well_name + '_calc_and_input' +  '.csv' )

Построение графиков для данных адаптации в папке  `adaptation`

Все данные (входные и результаты) на одном графике `well_name_calc_and_input.html`

In [ ]:
adapt_data_traces = pltl_wf.create_traces_list_for_all_columms(all_data, 'lines+markers', use_gl = True)
plot_file_path = path_to_work_dir + dir_name_with_calculated_data + well_name + '_calc_and_input' +  '.html'
pltl_wf.plot_subplots(adapt_data_traces, plot_file_path, True, auto_open = auto_open_html)

Создание отчета - набор нужных графиков для быстрой оценки результатов адаптации

In [ ]:
all_banches = pltl_opt.create_banches_for_report(report_type = 'adaptation')

pltl_wf.create_report_html(all_data, all_banches, path_to_work_dir + dir_name_with_calculated_data + 
                      well_name + '_adapt_report.html',  auto_open = auto_open_html)

Вывод обезразмеренных величин, тепловой карты для корреляции между параметрами, напорной характеристики

In [ ]:
static_data_path = preproc_tool.find_full_path_by_pattern(os.getcwd(), '*static_data.xlsx*')
static_data = workflow_tr_data.Static_data()
static_data_df = pd.read_excel(static_data_path[0])
static_data = workflow_tr_data.fill_static_data_structure_by_df(static_data, static_data_df, well_name + " (ready)")

In [ ]:
import unifloc_vba.description_generated.python_api as python_api
path_to_addin = os.getcwd()
path_to_addin = path_to_addin.replace('unifloc\\sandbox\\uTools', 'unifloc_vba\\UniflocVBA_7.xlam')
UniflocVBA = python_api.API(path_to_addin)
esp_traces = pltl_wf.create_esp_traces(UniflocVBA, static_data.esp_nom_rate_m3day, static_data.esp_nom_head_m, static_data.esp_id)

In [ ]:
esp_df = all_data[[global_names.dp_esp_atm + " (ADAPT)",
          global_names.esp_head_m + " (ADAPT)",
          global_names.efficiency_esp_d + " (ADAPT)",
          global_names.q_mix_mean_m3day + " (ADAPT)", 
          global_names.PowerESP_kwt + " (ADAPT)", 
          global_names.gas_fraction_intake_d + " (ADAPT)",
          global_names.rs_intake_m3m3 + " (ADAPT)",
          global_names.c_calibr_head_d + " (ADAPT)",
          global_names.c_calibr_power_d + " (ADAPT)"]]
esp_df = esp_df.set_index(esp_df[global_names.q_mix_mean_m3day + " (ADAPT)"])
esp_df_traces = pltl_wf.create_traces_list_for_all_columms(esp_df, chosen_mode = 'markers', use_gl = True)

In [ ]:
adapt_data_dimensionless = result_and_metrics.make_dimensionless_df(all_data)

In [ ]:
filename = path_to_work_dir + dir_name_with_calculated_data + well_name + '_dimless_pump_heatmap_report' + '.html'
pltl_wf.create_overall_report(all_data, adapt_data_dimensionless, esp_traces, filename, esp_df_traces,  auto_open = auto_open_html)

# 6 Загрузка данных адаптации для работы над восстановлением дебитов и прогнозом


In [ ]:
dir_name_with_input_data = 'adapt_input_' + '\\'
input_data_file_name = well_name + '_adapt_input'
dir_name_with_calculated_data = 'adaptation_' + '\\'
calculated_data_file_name = well_name + '_adapt_1'

In [ ]:
calculated_data = pd.read_csv(path_to_work_dir + dir_name_with_calculated_data + well_name + '_calc_and_input' +  '.csv', 
                             index_col = 'Время', parse_dates = True)

In [ ]:
calibr_data = calculated_data[[global_names.c_calibr_head_d + ' (ADAPT)',
                               global_names.c_calibr_power_d + ' (ADAPT)']]

In [ ]:
feature_data = calculated_data[[global_names.p_intake_atm, 
                               global_names.t_intake_c,
                               global_names.active_power_kwt, 
                               global_names.u_motor_v, 
                               global_names.p_buf_atm,
                               global_names.cos_phi_d,
                               global_names.freq_hz,
                               global_names.d_choke_mm]]

In [ ]:
calibr_data_without_index = calibr_data.reset_index()
feature_data_without_index = feature_data.reset_index()
calculated_data_without_index = calculated_data.reset_index()

# 6.1.1 Генерации данных для прогнозирования дебитов с помощью ML 

(back allocation, проверка back allocation)

Работа с калибровками для восстановления дебитов. Для преобразования их выделим в отдельный DataFrame. Будем интерполировать, выкалавать точки, либо еще что-нибудь

In [ ]:
calculated_data_x_train, calculated_data_x_test, calculated_data_y_train, calculated_data_y_test = calibr_restore.get_test_train_drop_2_points(calculated_data_without_index, calculated_data_without_index)

In [ ]:
x_train, x_test, y_train, y_test = calibr_restore.get_test_train_drop_2_points(feature_data_without_index, calibr_data_without_index)

In [ ]:
x_train, x_train_time = calibr_restore.extract_time_from_df(x_train)
x_test, x_test_time = calibr_restore.extract_time_from_df(x_test)
y_train, y_train_time = calibr_restore.extract_time_from_df(y_train)
y_test, y_test_time = calibr_restore.extract_time_from_df(y_test)

In [ ]:
head_calibr_pred = calibr_restore.restore_calibr_via_ridge(x_train, x_test, y_train[global_names.c_calibr_head_d + ' (ADAPT)']) 
power_calibr_pred = calibr_restore.restore_calibr_via_ridge(x_train, x_test, y_train[global_names.c_calibr_power_d + ' (ADAPT)'])
                                                                             
                                                                             



In [ ]:
y_test[global_names.c_calibr_head_d] = head_calibr_pred
y_test[global_names.c_calibr_power_d] = power_calibr_pred

In [ ]:
y_test[global_names.q_liq_m3day] = calculated_data_y_test[global_names.q_liq_m3day]
y_test[global_names.gor_m3m3] = calculated_data_y_test[global_names.gor_m3m3]
y_test[global_names.watercut_perc] = calculated_data_y_test[global_names.watercut_perc]
y_test[global_names.motor_load_perc] = calculated_data_y_test[global_names.motor_load_perc]

In [ ]:
y_test = y_test.set_index(y_test_time)
x_test = x_test.set_index(x_test_time)

In [ ]:
predict_input_data = x_test.join(y_test)

Генерация входных данных для модели для восстановления дебитов в папку `restore_input_time_mark`
* `well_name_restore_input.csv` - входные данных с калибровками (увеличили частоту дискретизации или выкололи точки)
* `well_name_restore_input.html` - все входные данные на одном графике

# 6.1.2 Генерации данных для прогнозирования дебитов с помощью ML 

forecast

Работа с калибровками для прогнозирования дебитов. Для преобразования их выделим в отдельный DataFrame. Будем интерполировать, выкалавать точки, либо еще что-нибудь

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate

In [ ]:
calculated_data_for_forecast = calculated_data.copy()

In [ ]:
calibr_data_for_forecast = calibr_data.copy()

In [ ]:
feature_data_for_forecast = feature_data.copy()

In [ ]:
def linear_forecast(x_true, y_true, x_forecast, power = 1):
    z = np.polyfit(x_true, y_true, 1)
    f = np.poly1d(z)
    y_forecast = f(x_forecast)
    return y_forecast

In [ ]:
def linear_forecast_with_time(df_input, parameter_to_forecast_str, n_ticks_to_learn, n_ticks_to_forecast, power = 1):
    df = df_input.copy()
    day_value = 86400000000000
    df.index = df.index.astype(int)
    
    border_to_learn = df.index[-1] - n_ticks_to_learn * day_value
    df_to_learn = df[df.index > border_to_learn]
    series_to_learn = df_to_learn[parameter_to_forecast_str]
    x_true = series_to_learn.index
    y_true = series_to_learn.values
    x_to_forecast = np.arange(x_true[-1], x_true[-1] + n_ticks_to_forecast * day_value, day_value)
    y_forecast = linear_forecast(x_true, y_true, x_to_forecast, power)
    x_forecast_as_time = pd.to_datetime(x_to_forecast)
    return x_forecast_as_time, y_forecast    

In [ ]:
forecasted_calculated_data = {}
n_ticks_to_learn = 3
n_ticks_to_forecast = 3
for i in global_names.return_essential_parameters():
    time_index, this_parameter_forecast = linear_forecast_with_time(calculated_data_for_forecast,
                                                        i, n_ticks_to_learn, n_ticks_to_forecast, power = 1)
    forecasted_calculated_data[i] = this_parameter_forecast
forecasted_calculated_data = pd.DataFrame(forecasted_calculated_data)
forecasted_calculated_data.index = time_index
forecasted_calculated_data.tail()

In [ ]:
forecasted_calculated_data[global_names.d_choke_mm] = forecasted_calculated_data[global_names.p_intake_atm] * 0 + 32

In [ ]:
x_train = calculated_data_for_forecast[[global_names.p_intake_atm, 
                               global_names.t_intake_c,
                               global_names.active_power_kwt, 
                               global_names.u_motor_v, 
                               global_names.p_buf_atm,
                               global_names.cos_phi_d,
                               global_names.freq_hz,
                               global_names.d_choke_mm]].reset_index(drop = True)


In [ ]:
y_train = calculated_data_for_forecast[[global_names.c_calibr_head_d + ' (ADAPT)', 
                                        global_names.c_calibr_power_d + ' (ADAPT)']].reset_index(drop = True)

In [ ]:
x_test = forecasted_calculated_data[[global_names.p_intake_atm, 
                               global_names.t_intake_c,
                               global_names.active_power_kwt, 
                               global_names.u_motor_v, 
                               global_names.p_buf_atm,
                               global_names.cos_phi_d,
                               global_names.freq_hz,
                               global_names.d_choke_mm]].reset_index(drop = True)

In [ ]:
head_calibr_pred = calibr_restore.restore_calibr_via_ridge(x_train, x_test, y_train[global_names.c_calibr_head_d + ' (ADAPT)']) 
power_calibr_pred = calibr_restore.restore_calibr_via_ridge(x_train, x_test, y_train[global_names.c_calibr_power_d + ' (ADAPT)'])
                                                                             
                          
        

In [ ]:
predict_input_data = forecasted_calculated_data.copy()
predict_input_data[global_names.c_calibr_head_d] = head_calibr_pred
predict_input_data[global_names.c_calibr_power_d] = power_calibr_pred
predict_input_data.index = forecasted_calculated_data.index
predict_input_data.index.name = 'Время'

# 6.2 Построение графиков


In [ ]:
predict_input_data = preproc_tool.fill_input_data(predict_input_data, global_names.return_essential_parameters())

In [ ]:
input_data_dir_name = 'restore_input_' + time_mark
path_to_input_data = path_to_work_dir + input_data_dir_name + '\\'
if create_input_data:
    try:
        os.mkdir(path_to_work_dir + input_data_dir_name)
    except:
        pass

In [ ]:
predict_input_data.to_csv(path_to_input_data + well_name + '_restore_input.csv')

In [ ]:
plot_file_path = path_to_input_data + well_name + '_restore_input.html'
input_data_traces = pltl_wf.create_traces_list_for_all_columms(predict_input_data, 'lines+markers', use_gl = True)
pltl_wf.plot_subplots(input_data_traces, plot_file_path, True,  auto_open = auto_open_html)

Построение графиков в отчетной форме 

In [ ]:
all_banches = pltl_opt.create_banches_for_report(report_type = 'forecast_input')

pltl_wf.create_report_html(predict_input_data, all_banches, path_to_input_data  + 
                      well_name + '_restore_input_report.html',  auto_open = auto_open_html)

# 7. Восстановление дебитов.
Нужно, используя сгенерированные данные `well_name_restore_input` восстановить дебиты с помощью `postprocessor.py`
Для этого активировать флаги
* vfm_calc_option = True
* restore_q_liq_only = True

Подготовка к параллельному расчету.

В проекте UniflocVBA нужно вручную размножить надстройки до нужного количества потоков - каждая надстройка будет работать параллельно и рассчитывать определенную часть общих данных. 

По умолчанию выставлено 4 потока на 4 надстройках `UniflocVBA_7.xlam`, `UniflocVBA_7_1.xlam`, `UniflocVBA_7_2.xlam`, `UniflocVBA_7_3.xlam`. При желании можно добавить еще, не забыв изменить также номера потоков и их общее количество.

Группировка информации о потоках в единый список `thread_option_list`

In [ ]:
# настройка многопоточности

amount_of_threads = 1
dir_name_with_input_data = 'restore_input_'

thread_option_list =proc.create_thread_list(well_name, dir_name_with_input_data, static_data_full_path,
                       amount_of_threads)

Запуск процесса восстановления дебитов. Процесс может быть долгим - в среднем расчет восстановления дебитов идет быстрее, чем при адаптации (получении калибровок). 

Рассчет 1 месяца в среднем занимает около 10-15 минут.

Стоит отметить, что для аварийного выхода нужно вручную закрыть надстройки Excel (окна, которые открыты и в них ведется работа). После закрытия всех четырех процесс остановится и возникнет ошибка, контроль над jupyter notebook вернется и можно будет работать.

Поэтому для тщательной отладки processor.py рекомендуется его запускать отдельно.

In [ ]:
#%%time
run_calculation(thread_option_list)

Результаты расчета появятся в `well_name/restore_time_mark/multiprocessing`

# 8. Подведение итогов

## 8.1 Первичная обработка результатов прогноза
Обозначим директории с результатами восстановления дебитов и входными данными

In [ ]:
dir_name_with_input_data = 'restore_input_' + '\\'
input_data_file_name = well_name +'_restore_input'
dir_name_with_calculated_data = 'restore_' + '\\'
calculated_data_file_name = well_name +'_restore_1'

Объединение данных при запуске расчета в многопотоке. Если расчет проводился без многопоточности, этот шаг можно пропустить.

In [ ]:
if multiprocessing_on == True:
    first_result_data = preproc_tool.combine_multiprocessing_result(path_to_work_dir, dir_name_with_calculated_data)
    first_result_data.to_csv(path_to_work_dir + dir_name_with_calculated_data + calculated_data_file_name + '.csv')

Загрузка данных после восстановления дебитов

In [ ]:
calculated_data = workflow_calc_data.load_calculated_data_from_csv(path_to_work_dir + dir_name_with_calculated_data +
                                                calculated_data_file_name +  '.csv', "PREDICTION")

Объединение входных и выходных данным модели. Сохранение результатов в `well_name_calc_and_input.csv`

In [ ]:
input_data = pd.read_csv(path_to_work_dir + dir_name_with_input_data + input_data_file_name +  '.csv', parse_dates = True, index_col = 'Время')
all_data = input_data.join(calculated_data, how = 'outer')

In [ ]:
if pd.isna(all_data.iloc[0][0]):
    print('Удаление пустой первой строки')
    all_data = all_data.drop(index = all_data.index[0])

In [ ]:
all_data.to_csv(path_to_work_dir + dir_name_with_calculated_data + well_name + '_calc_and_input' +  '.csv' )

Построение графиков восстановления дебитов

In [ ]:
input_data_traces = pltl_wf.create_traces_list_for_all_columms(all_data, 'lines+markers', use_gl = True)
plot_file_path = path_to_work_dir + dir_name_with_calculated_data + well_name + '_calc_and_input' +  '.html'
pltl_wf.plot_subplots(input_data_traces, plot_file_path, True,  auto_open = auto_open_html)

In [ ]:
all_banches = pltl_opt.create_banches_for_report(report_type = 'prediction')

pltl_wf.create_report_html(all_data, all_banches, path_to_work_dir  + 
                      dir_name_with_calculated_data + well_name + '_prediction_report.html',  auto_open = auto_open_html)

In [ ]:
all_banches = pltl_opt.create_banches_for_report(report_type = 'forecast')

pltl_wf.create_report_html(all_data, all_banches, path_to_work_dir  + 
                      dir_name_with_calculated_data + well_name + '_prediction_report.html',  auto_open = auto_open_html)

In [ ]:
static_data_path = preproc_tool.find_full_path_by_pattern(os.getcwd(), '*static_data.xlsx*')
static_data = workflow_tr_data.Static_data()
static_data_df = pd.read_excel(static_data_path[0])
static_data = workflow_tr_data.fill_static_data_structure_by_df(static_data, static_data_df, well_name + " (ready)")

In [ ]:
import unifloc_vba.description_generated.python_api as python_api
path_to_addin = os.getcwd()
path_to_addin = path_to_addin.replace('unifloc\\sandbox\\uTools', 'unifloc_vba\\UniflocVBA_7.xlam')
UniflocVBA = python_api.API(path_to_addin)
esp_traces = pltl_wf.create_esp_traces(UniflocVBA, static_data.esp_nom_rate_m3day, static_data.esp_nom_head_m, static_data.esp_id)

In [ ]:
esp_df = all_data[[global_names.dp_esp_atm + " (PREDICTION)",
          global_names.esp_head_m + " (PREDICTION)",
          global_names.efficiency_esp_d + " (PREDICTION)",
          global_names.q_mix_mean_m3day + " (PREDICTION)", 
          global_names.PowerESP_kwt + " (PREDICTION)", 
          global_names.gas_fraction_intake_d + " (PREDICTION)",
          global_names.rs_intake_m3m3 + " (PREDICTION)",
          global_names.c_calibr_head_d + " (PREDICTION)",
          global_names.c_calibr_power_d + " (PREDICTION)"]]
esp_df = esp_df.set_index(esp_df[global_names.q_mix_mean_m3day + " (PREDICTION)"])
esp_df_traces = pltl_wf.create_traces_list_for_all_columms(esp_df, chosen_mode = 'markers', use_gl = True)

In [ ]:
prediction_data_dimensionless = result_and_metrics.make_dimensionless_df(all_data)

In [ ]:
filename = path_to_work_dir + dir_name_with_calculated_data + well_name + '_dimless_pump_heatmap_report' + '.html'
pltl_wf.create_overall_report(all_data, prediction_data_dimensionless, esp_traces, filename, esp_df_traces,  auto_open = auto_open_html,
                             mark = " (PREDICTION)")

## 8.2 Сведение данных адаптации и восстановления
Использование файлов типа `calc_and_input` в папках с адаптацией и восстновлением для формирования общего отчета

In [ ]:
path_to_adapt_dir = 'adaptation_' + '\\'
path_to_restore_dir = 'restore_' + '\\'
dir_name_with_calculated_data = 'restore_' + '\\'
calculated_data_file_name = well_name +'_restore_1'

Загрузка и слияние данных адаптации и восстановления

In [ ]:
adapt_data_with_input = pd.read_csv(path_to_work_dir + path_to_adapt_dir + well_name + '_calc_and_input' + '.csv' , parse_dates = True, index_col = 'Время')
restore_data_with_input = pd.read_csv(path_to_work_dir + path_to_restore_dir + well_name + '_calc_and_input' + '.csv' , parse_dates = True, index_col = 'Время')
overall_data = adapt_data_with_input.join(restore_data_with_input, how = 'outer', lsuffix = '', rsuffix = ' (PREDICT double)')

Заключительная обработка данных

In [ ]:
overall_data = result_and_metrics.add_relative_errors_to_overall_data(overall_data)

Подготовка вывода в отчет

In [ ]:
all_banches = pltl_opt.create_banches_for_report(report_type = 'overall_result')

Создание сводного отчета `well_name_adapt_and_restore_report.html`

In [ ]:
input_data_traces = pltl_wf.create_traces_list_for_all_columms(overall_data, 'lines+markers', use_gl = True)
plot_file_path = path_to_work_dir + dir_name_with_calculated_data + well_name + '_adapt_and_restore' +  '.html'
pltl_wf.plot_subplots(input_data_traces, plot_file_path, True,  auto_open = auto_open_html)

In [ ]:
plot_file_path = path_to_work_dir + path_to_restore_dir + well_name + '_adapt_and_restore_report' +  '.html'
pltl_wf.create_report_html(overall_data, all_banches, plot_file_path,  auto_open = auto_open_html)

Расчет различных метрик для модели и сохранение их в текстовый файл `well_name_adapt_and_restore_metrics_report.txt`

In [ ]:
result = result_and_metrics.make_result(overall_data)

In [ ]:
result_file_path = path_to_work_dir + path_to_restore_dir + well_name + '_adapt_and_restore_metrics_report' +  '.xlsx'
result.to_excel(result_file_path)

# Конец